In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path={'executable_path':ChromeDriverManager().install()}
browser=Browser('chrome',**executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\jlubr\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [3]:
# Chathams Bottlekings wine url

url='https://chatham.bottleking.com/main.asp?request=search&type=w'
browser.visit(url)

In [4]:
#Dataframe where all the data will be staged before writing it into a .csv file.

wine_df=pd.DataFrame(columns=['merchant','address','name','regprice','description','country',
                              'region','subregion','variety','type'])

merchant='Chatham BottleKing'
address='41 Watchung Ave, Chatham, NJ, 07928'

print("Scraping please be patient")
while True:
    html=browser.html
    soup=BeautifulSoup(html,'html.parser')
    
# Total wines on the website.
    wines=soup.find_all('a',{'class':['Srch-producttitle']})

# Sku's of all the wines which will be used to create a new url so that further details
#     can be obtained.
    wines_sku=soup.find_all('span',{'class':['Srch-sku']}) 

# Descriptions of all the wines..
    wines_description=soup.find_all('span',{'class':['Srch-partialDesc']})
    wines_bottleprice=soup.find_all('span',{'class':['RegularPrice']})

# Click into details of a wine for the length of all wines on the website.
    for i in range(len(wines)):
        sku=wines_sku[i].text.replace('Sku: ','')

# Detail url for the wine.
        url='https://chatham.bottleking.com/sku'+ sku +'.html'
        try:

# All data was not available for all wines.  Some wines had subregion and some did not..
            subregion='N/A'

# Visit detail of wine bottle using the new url..
            browser.visit(url)
            html=browser.html
            soup=BeautifulSoup(html,'html.parser')
            more_info=str(soup.find_all('a',{'class':['reglink']}))
#            print(more_info)
            country=more_info.split('country=')[1]
            country=country.split('">')[0]
            region=more_info.split('region=')[1]
            region=region.split('">')[0]
            variety=more_info.split('variety=')[1]
            variety=variety.split('">')[0]
            wtype=more_info.split('type=W">')[1]
            wtype=wtype.split('<')[0]

# Condition for if this wine has a subregion or not...
            if (more_info.find('subregion')!=-1):
                subregion=more_info.split('subregion=')[1]
                subregion=subregion .split('">')[0]

# Return to previous page..
            browser.back()
        except:

# If wine has no detail the all fields are N/A and return to previous page..
            country='N/A'
            region='N/A'
            subregion='N/A'
            variety='N/A'
            wtype='N/A'
            browser.back()

# Append to dataframe..
        wine_df=wine_df.append({
            'merchant':merchant,
            'address':address,
            'name':str(wines[i].text),
            'regprice':str(wines_bottleprice[i].text),
            'description':str(wines_description[i].text),
            'country':country,
            'region':region,
            'subregion':subregion,
            'variety':variety,
            'type':wtype
        },ignore_index=True)       

# When going through all the wines on a page click to the next page of wines.. 
#     otherwise if there are no more pages, break out of while loop and end the scraping
#     and write the dataframe obtained into a csv file..
    try:
        browser.links.find_by_partial_text('>>').click()
    except:
        break
browser.quit()
print('Scraping Complete')
wine_df.to_csv("chathambuyrite.csv")

Scraping please be patient
Scraping Complete
